In [8]:
import csv
import pandas as pd
import numpy as np
import sklearn.utils

from sklearn.feature_extraction.text import CountVectorizer

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, utils
import torch.nn.functional as F
import torch.nn.init as torch_init

In [2]:
# Get dataframe and simple pared down version of it
training_dataframe = sklearn.utils.shuffle(pd.read_csv("train-balanced-sarcasm.csv")).iloc[:250]

# Create vectorizer to convert words to vectors
vectorizer = CountVectorizer()
vectorizer.fit(training_dataframe['comment'].values.astype('U'))

# Create training data and labels with dataframe and vectorizer
y = np.matrix(training_dataframe["label"].tolist()).T
x = np.matrix(vectorizer.transform(training_dataframe['comment'].values.astype('U')).toarray())

In [7]:
np.split(x, 1)

[matrix([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])]

In [14]:
# Dimensions of input for our model
input_dim = x.shape[1]

# Dimension of middle layers (we may not use all three)
mid1 = input_dim
mid2 = input_dim // 2
mid3 = input_dim // 4

output_dim = y.shape[1]

# Simple neural network which takes a vectorized bag of words as input
class BowNN(nn.Module):  # inheriting from nn.Module!

    # TODO: Improve this by passing in parameters instead of using global variables
    def __init__(self):

        super(BowNN, self).__init__()
        
        self.tanh = nn.Tanh();
        self.sigm = nn.Sigmoid();

        self.fc1 = nn.Linear(input_dim, mid1)
        torch_init.xavier_normal_((self.fc1.weight))
        
        self.fc2 = nn.Linear(mid1, mid3)
        torch_init.xavier_normal_((self.fc2.weight))
        
        self.fc3 = nn.Linear(mid3, output_dim) 
        torch_init.xavier_normal_((self.fc3.weight))


    def forward(self, bow_vec):
        # Pass input through multiple layers
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.sigm(out)
        
        return out

    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet().to(device)

    
# Use binary cross entropy loss
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)


In [11]:
y.shape[1]

1